In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Importing necessary Libraries**

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import models,datasets,transforms
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Setting GPU

In [ ]:
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#print(device)

# Setting Data Transformer for pre-processsing images

In [ ]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Importing datasets and setting up dataloader

In [ ]:
test_path = '/content/drive/MyDrive/assignment1_data/classification_dataset/test'
train_path = '/content/drive/MyDrive/assignment1_data/classification_dataset/train'
train_data = datasets.ImageFolder(root = train_path,transform=transform,target_transform=None)
test_data = datasets.ImageFolder(root = test_path,transform=transform,target_transform=None)
train_dataloader = DataLoader(dataset = train_data,batch_size=1,shuffle=True)
test_dataloader = DataLoader(dataset = test_data,batch_size=1,shuffle=False)

# Loading Pre-Trained Model(ResNet-18)

In [ ]:
resnet18 = models.resnet18(pretrained=True)
resnet18=nn.Sequential(*list(resnet18.children())[:-1])
#resnet18 = resnet18.to(device)
resnet18.eval()

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

# Question (a)

# Extracting features from Training and Testing Data

In [ ]:
i=0
for images,labels in train_dataloader:
  #images = images.to(device)
  #labels = labels.to(device)
  features = resnet18(images).squeeze()
  features = features.detach().numpy()
  if i==0:
    x_train = features
    y_train = labels.detach().numpy()
    i=1
  else:
    x_train = np.row_stack((x_train,features))
    y_train = np.row_stack((y_train,labels))



In [ ]:
i=0
for images,labels in test_dataloader:
  #images = images.to(device)
  #labels = labels.to(device)
  features = resnet18(images).squeeze()
  features = features.detach().numpy()
  if i==0:
    x_test = features
    y_test = labels.detach().numpy()
    i=1
  else:
    x_test = np.row_stack((x_test,features))
    y_test = np.row_stack((y_test,labels))

# Using k-NN to classify

In [ ]:
# Define the number of nearest neighbors (k)
k = 5

# Create the k-NN classifier
knn = KNeighborsClassifier(n_neighbors=k)

# Fit the k-NN classifier to the features and labels
knn.fit(x_train, y_train.ravel())

# Predict the labels of the test images
predicted_labels = knn.predict(x_test)

# Evaluate the accuracy of the classifier
accuracy = accuracy_score(y_test.ravel(), predicted_labels)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 96.67%


# Question (b)

In [ ]:
train_dataloader_1 = DataLoader(dataset = train_data,batch_size=64,shuffle=True)
test_dataloader_1 = DataLoader(dataset = test_data,batch_size=1,shuffle=False)

In [ ]:
!pip install -q torchinfo
from torchinfo import summary

# Initializing model

In [ ]:
resnet18_1 = models.resnet18(pretrained=True)
# Freezing weights of layers
for param in resnet18_1.parameters():
     param.requires_grad=False
# Replace the ImageNet classifier with a new classifier for your dataset
num_classes = 6
resnet18_1.fc = nn.Linear(resnet18_1.fc.in_features, num_classes)

# Fine-tuning classification layer using training data


In [ ]:
learning_rate = 0.001
num_epochs = 30
loss_fun = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet18_1.parameters(), lr=learning_rate)

In [ ]:
# Fine-tune the model
running_loss = 0
for epoch in range(num_epochs):
    for imgs, labels in train_dataloader_1:
        # Move the data to the #GPU
        #imgs = imgs.to(device)
        #labels = labels.to(device)

        # Forward pass
        outputs = resnet18_1(imgs)
        loss = loss_fun(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss = running_loss + loss.item()

    # Print the current loss
    print("Epoch {}/{}, Loss: {:.4f}".format(epoch+1, num_epochs, running_loss/(epoch+1)))

Epoch 1/30, Loss: 0.2012
Epoch 2/30, Loss: 0.1837
Epoch 3/30, Loss: 0.1795
Epoch 4/30, Loss: 0.1723
Epoch 5/30, Loss: 0.1641
Epoch 6/30, Loss: 0.1607
Epoch 7/30, Loss: 0.1512
Epoch 8/30, Loss: 0.1446
Epoch 9/30, Loss: 0.1465
Epoch 10/30, Loss: 0.1417
Epoch 11/30, Loss: 0.1375
Epoch 12/30, Loss: 0.1377
Epoch 13/30, Loss: 0.1349
Epoch 14/30, Loss: 0.1317
Epoch 15/30, Loss: 0.1286
Epoch 16/30, Loss: 0.1257
Epoch 17/30, Loss: 0.1233
Epoch 18/30, Loss: 0.1278
Epoch 19/30, Loss: 0.1354
Epoch 20/30, Loss: 0.1357
Epoch 21/30, Loss: 0.1356
Epoch 22/30, Loss: 0.1338
Epoch 23/30, Loss: 0.1326
Epoch 24/30, Loss: 0.1315
Epoch 25/30, Loss: 0.1299
Epoch 26/30, Loss: 0.1274
Epoch 27/30, Loss: 0.1347
Epoch 28/30, Loss: 0.1332
Epoch 29/30, Loss: 0.1344
Epoch 30/30, Loss: 0.1330


In [ ]:
#Setting evaluation mode
resnet18_1.eval()
# Testing on test dataset
count=0
total=0
for imgs,labels in test_dataloader_1:
  output = resnet18_1(imgs)
  pred = torch.argmax(output)
  if pred == labels :
    count = count+1
  total=total+1
print('The percentage accuracy is')
print(count/total *100)

The percentage accuracy is
100.0


# Question (c)

In [ ]:
class Simple_linear_network(nn.Module):
  def __init__(self):
    super(Simple_linear_network,self).__init__()
    self.layer1 = nn.Flatten()
    self.layer2 = nn.Linear(150528,6)
  def forward(self,x):
    x=self.layer1(x)
    x=self.layer2(x)
    return(x)


In [ ]:
simple_model = Simple_linear_network()
simple_model

Simple_linear_network(
  (layer1): Flatten(start_dim=1, end_dim=-1)
  (layer2): Linear(in_features=150528, out_features=6, bias=True)
)

# Training the model

In [ ]:
loss_fun = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(simple_model.parameters(), lr=0.001)
running_loss = 0
for epoch in range(num_epochs):
    for imgs, labels in train_dataloader_1:
        # Move the data to the #GPU
        #imgs = imgs.to(device)
        #labels = labels.to(device)

        # Forward pass
        outputs = simple_model(imgs)
        loss = loss_fun(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss = running_loss + loss.item()

    # Print the current loss
    print("Epoch {}/{}, Loss: {:.4f}".format(epoch+1, num_epochs, running_loss/(epoch+1)))

Epoch 1/30, Loss: 19.8864
Epoch 2/30, Loss: 23.4889
Epoch 3/30, Loss: 25.7035
Epoch 4/30, Loss: 22.6645
Epoch 5/30, Loss: 23.0216
Epoch 6/30, Loss: 21.6010
Epoch 7/30, Loss: 21.5256
Epoch 8/30, Loss: 20.0922
Epoch 9/30, Loss: 18.9157
Epoch 10/30, Loss: 17.8478
Epoch 11/30, Loss: 16.8147
Epoch 12/30, Loss: 15.6506
Epoch 13/30, Loss: 14.6266
Epoch 14/30, Loss: 14.0583
Epoch 15/30, Loss: 13.5111
Epoch 16/30, Loss: 12.9608
Epoch 17/30, Loss: 12.6076
Epoch 18/30, Loss: 12.0921
Epoch 19/30, Loss: 11.7083
Epoch 20/30, Loss: 11.7153
Epoch 21/30, Loss: 11.6361
Epoch 22/30, Loss: 11.9440
Epoch 23/30, Loss: 11.8516
Epoch 24/30, Loss: 11.5728
Epoch 25/30, Loss: 11.2571
Epoch 26/30, Loss: 10.9331
Epoch 27/30, Loss: 10.6100
Epoch 28/30, Loss: 10.2812
Epoch 29/30, Loss: 9.9945
Epoch 30/30, Loss: 9.7213


In [ ]:
#Setting evaluation mode
simple_model.eval()
# Testing on test dataset
count=0
total=0
for imgs,labels in test_dataloader_1:
  output = simple_model(imgs)
  pred = torch.argmax(output)
  if pred == labels :
    count = count+1
  total=total+1
print('The percentage accuracy is')
print(count/total *100)

The percentage accuracy is
32.5
